In [2]:
import pandas as pd
import yfinance as yf
import numpy as np

#These functions are for dealing with multiple stocks. 

def list_of_SP500_stocks_info(URL: str) -> pd.DataFrame:

    tickers = pd.read_html(URL)[0]['Symbol'].tolist()
    industry = pd.read_html(URL)[0]['GICS Sector'].tolist()

    data = {
    'Ticker': tickers,
    'Industry': industry
    }

    df = pd.DataFrame(data)
    
    return df, tickers


def create_stock_df(tickers: list) -> pd.DataFrame:

    df = yf.download(tickers, start='2022-02-01', end='2023-02-01')

    df = df[['Open', 'Adj Close', 'High', 'Low']]

    df = _calculate_SMA(df)

    df = df.T

    df.index.set_names(['Metric','Ticker'], inplace=True)

    df = df.swaplevel(0, 1)
    
    df.sort_index(axis=1, level=0, inplace=True)

    return df


def _calculate_SMA(df: pd.DataFrame):

    # create 30 day SMA
    sma30_df = df['Adj Close'].rolling(30).mean()
    sma30_label = "SMA30"
    sma30_df.columns = pd.MultiIndex.from_product([[sma30_label],sma30_df.columns.unique()])

    # create 200 day SMA
    sma200_df = df['Adj Close'].rolling(200).mean()
    sma200_label = "SMA200"
    sma200_df.columns = pd.MultiIndex.from_product([[sma200_label],sma200_df.columns.unique()])

    # concatenate to back to main df
    final_df = pd.concat([df, sma30_df, sma200_df],axis=1)

    return final_df

def _join_industry_info_to_stock_data(all_stocks_df: pd.DataFrame, stock_info_df: pd.DataFrame) -> pd.DataFrame:
    
    joined_df = pd.merge(all_stocks_df.reset_index(), stock_info_df.reset_index(), on=['Ticker'], how='inner').set_index(['Industry','Ticker','Metric'])
    joined_df = joined_df.sort_index()
    joined_df = joined_df.drop('index', axis=1)

    return joined_df


In [4]:

df = yf.download(['AAPL', 'MSFT'], start='2022-02-01', end='2023-02-01')

df = df[['Open', 'Adj Close', 'High', 'Low']]

df



[*********************100%***********************]  2 of 2 completed


Open               Adj Close                    High  \
                  AAPL        MSFT        AAPL        MSFT        AAPL   
Date                                                                     
2022-02-01  174.009995  310.410004  173.337814  305.129456  174.839996   
2022-02-02  174.750000  309.630005  174.558823  309.774170  175.880005   
2022-02-03  174.479996  309.489990  171.640259  297.707794  176.240005   
2022-02-04  171.679993  300.209991  171.351990  302.342651  174.100006   
2022-02-07  172.860001  306.170013  170.626389  297.411285  173.949997   
...                ...         ...         ...         ...         ...   
2023-01-25  140.889999  234.479996  141.643738  240.008850  142.429993   
2023-01-26  143.169998  243.649994  143.740540  247.380386  144.250000   
2023-01-27  143.160004  248.990005  145.707520  247.539993  147.229996   
2023-01-30  144.960007  244.509995  142.781998  242.103607  145.550003   
2023-01-31  142.699997  243.449997  144.070023  247.190857  144.339996   

                               Low              
                  MSFT        AAPL        MSFT  
Date                                            
2022-02-01  310.630005  172.309998  305.130005  
2022-02-02  315.119995  173.330002  308.880005  
2022-02-03  311.230011  172.119995  299.959991  
2022-02-04  308.799988  170.679993  299.970001  
2022-02-07  307.839996  170.949997  299.899994  
...                ...         ...         ...  
2023-01-25  243.300003  138.809998  230.899994  
2023-01-26  248.309998  141.899994  242.000000  
2023-01-27  249.830002  143.080002  246.830002  
2023-01-30  245.600006  142.850006  242.199997  
2023-01-31  247.949997  142.279999  242.949997  

[251 rows x 8 columns]

In [ ]:
df = _calculate_SMA(df)
df